In [ ]:
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, AutoencoderKL, PNDMScheduler
from transformers import T5EncoderModel, T5TokenizerFast
import torch
import os
import numpy as np
import librosa
from tqdm import tqdm
import soundfile as sf
import torchaudio
from IPython.display import Audio
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print(torch.__version__)

In [ ]:
# model_path = "/blob/v-yuancwang/AudioEditingModel/Diffusion_GEN/checkpoint-16000"
model_path = "/blob/v-yuancwang/AudioEditingModel/Diffusion_SG/checkpoint-42000"
vae = AutoencoderKL.from_pretrained(model_path, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(model_path, subfolder="unet")
tokenizer = T5TokenizerFast.from_pretrained(model_path, subfolder="tokenizer")
text_encoder = T5EncoderModel.from_pretrained(model_path, subfolder="text_encoder")

In [ ]:
torch_device = "cuda:2"
vae.to(torch_device)
text_encoder.to(torch_device)
unet.to(torch_device)
vae.requires_grad_(False)
unet.requires_grad_(False)
text_encoder.requires_grad_(False)

In [ ]:
save_mel_path = "/home/v-yuancwang/AudioEditing/test_generate"
save_fig_path = "/home/v-yuancwang/AudioEditing/figs"
# mel_src = np.load("/blob/v-yuancwang/audio_editing_data/sr/mel/LsCTeF3TbAU.npy")
# mel_tgt = np.load("/blob/v-yuancwang/audio_editing_data/audiocaps/mel/LsCTeF3TbAU.npy")
text = "Generate: jazz music"
# np.save(os.path.join(save_mel_path, text.replace(":", " ") + "_src.npy"), mel_src)
# np.save(os.path.join(save_mel_path, text.replace(":", " ") + "_tgt.npy"), mel_tgt)

In [ ]:
plt.imshow(mel_src)
plt.savefig(fname=os.path.join(save_fig_path, text.replace(":", " ") + "_src.png"))

In [ ]:
plt.imshow(mel_tgt)
plt.savefig(fname=os.path.join(save_fig_path, text.replace(":", " ") + "_tgt.png"))

In [ ]:
prompt = [text]
text_input = tokenizer(prompt, max_length=tokenizer.model_max_length, truncation=True, padding="do_not_pad", return_tensors="pt")
text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * 1, padding="max_length", max_length=max_length, return_tensors="pt"
)
uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0] 
print(text_embeddings.shape)
print(uncond_embeddings.shape)
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

In [ ]:
latents_src = torch.Tensor(np.array([[mel_src]])).to(torch_device)
latents_src = vae.encode(latents_src).latent_dist.sample()
print(latents_src.shape, latents_src.dtype, latents_src.device)

In [ ]:
from diffusers import DDIMScheduler
num_inference_steps = 100
# scheduler = DDIMScheduler.from_pretrained(model_path, subfolder="scheduler")
scheduler = PNDMScheduler.from_pretrained(model_path, subfolder="scheduler")
scheduler.set_timesteps(num_inference_steps)

In [ ]:
guidance_scale = 7.5
scheduler.set_timesteps(num_inference_steps)

latents = torch.randn((1, 4, 10, 78)).to(torch_device)
# latents_src_input = torch.cat([latents_src] * 2)

for t in tqdm(scheduler.timesteps):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)

    latent_model_input = scheduler.scale_model_input(latent_model_input, timestep=t)

    # predict the noise residual
    with torch.no_grad():
        # noise_pred = unet(torch.cat((latent_model_input, latents_src_input), dim=1), t, encoder_hidden_states=text_embeddings).sample
        # noise_pred = unet(torch.cat((latent_model_input, latent_model_input), dim=1), t, encoder_hidden_states=text_embeddings).sample
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
    
    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred, t, latents).prev_sample

In [ ]:
# latents_out = latents_src + latents
latents_out = latents

In [ ]:
with torch.no_grad():
    res = vae.decode(latents_out).sample
res = res.cpu().numpy()[0,0,:,:]

In [ ]:
np.save(os.path.join(save_mel_path, text.replace(":", " ") + ".npy"), res)
plt.imshow(res)
plt.xticks([])
plt.yticks([])
plt.savefig(fname=os.path.join(save_fig_path, text.replace(":", " ") + ".png"))